In [36]:
import collections
import os
import random
import urllib
import zipfile

import numpy as np 
import tensorflow as tf

In [37]:
# Training Parameters.
learning_rate = 0.1
batch_size = 128
num_steps = 3000000
display_step = 10000
eval_step = 200000

# Evaluation Parameters.
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

# Word2Vec Parameters.
embedding_size = 200 # Dimension of the embedding vetor.
max_vocabulary_size = 50000 # Total number of different words in the vocabulary
min_occurrence = 10 # Remove all words that does not appears at least n times.
skip_window = 3 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
num_sampled = 64 # Number of negative examples to sample.

In [38]:
# Download a small chunk of Wikipedia articles collection.
url = 'http://mattmahoney.net/dc/text8.zip'
data_path = 'text8.zip'
if not os.path.exists(data_path):
  print("Downloading the dataset... (It may take some time)")
  filename, _ = urllib.request.urlretrieve(url, data_path)
  print("Done!")
# Unzip the dataset file. Text has already been processed.
with zipfile.ZipFile(data_path) as f:
  text_words = f.read(f.namelist()[0]).lower().split()

In [39]:
# Build the dictionary and replace rare words with UNK token.
count = [('UNK', -1)]
# Retrieve the most common words.
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
# Remove samples with less than 'min_occurrence' occurrences.
for i in range(len(count) - 1, -1, -1):
  if count[i][1] < min_occurrence:
    count.pop(i)
  else:
    # The collection is ordered, so stop when 'min_occurrence' is reached.
    break

# Compute the vocabulary size.
vocabulary_size = len(count)
# Assign an id to each word.
word2id = dict()
for i, (word, _) in enumerate(count):
  word2id[word] = i

data = list()
unk_count = 0
for word in text_words:
  # Retrive a word id, or assign it index 0 ('UNK') if not in dictionary
  index = word2id.get(word, 0)
  if index == 0:
    unk_count += 1
  data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Words count: 17005207
Unique words: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), (b'the', 1061396), (b'of', 593677), (b'and', 416629), (b'one', 411764), (b'in', 372201), (b'a', 325873), (b'to', 316376), (b'zero', 264975), (b'nine', 250430)]


In [40]:
data_index = 0
# Generate training batch for the skip-gram model.
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one).
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch.
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [41]:
# Ensure the following ops & var are assigned on CPU
# (some ops are not compatible on GPU).
with tf.device('/cpu:0'):
  # Create the embedding variable (each row represent a word embedding vector).
  embedding = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
  # Construct the variables for the NCE loss.
  nce_weights = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
  nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

def get_embedding(x):
  with tf.device('/cpu:0'):
    # Lookup the corresponding embedding vectors for each sample in X.
    x_embed = tf.nn.embedding_lookup(embedding, x)
    return x_embed

def nce_loss(x_embed, y):
  with tf.device('/cpu:0'):
    # Compute the average NCE loss for the batch.
    y = tf.cast(y, tf.int64)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights,
                       biases=nce_biases,
                       labels=y,
                       inputes=x_embed,
                       num_sampled=num_sampled,
                       num_classes=vocabulary_size)
    )
    return loss

# Evaluation
def evaluate(x_embed):
  with tf.device('/cpu:0'):
    # Compute the cosine similarity between input data embedding and every embedding vectors
    x_embed = tf.cast(x_embed, tf.float32)
    x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
    embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdim=True))
    cosine_sim_op = tf.matmul(x_embed_norm, embedding_norm, transpose_b=True)
    return cosine_sim_op

# Define the optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [42]:
# Optimization process
def run_optimization(x, y):
  with tf.device('/cpu:0'):
    # Wrap computation inside a GradientTape for automatic differentiation
    with tf.GradientTape() as g:
      emb = get_embedding(x)
      loss = nce_loss(emb, y)

    # Compute gradient.
    gradients = g.gradient(loss, [embedding, nce_weights, nce_biases])

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [embedding, nce_weights, nce_biases]))

In [ ]:
# change word2id dict's key code
new_word2id = {}
for (key, value) in word2id.items():
  if type(key) is bytes:
    print(key)
    new_word2id[key.decode('utf-8')] = value
  else:
    print(key, type(key))
    new_word2id[key] = value

In [74]:
# Words for testing.
x_test = np.array([new_word2id[w] for w in eval_words])

# Run training for the given number of steps.
for step in range(1, num_steps + 1):
    batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
    run_optimization(batch_x, batch_y)
    
    if step % display_step == 0 or step == 1:
        loss = nce_loss(get_embedding(batch_x), batch_y)
        print("step: %i, loss: %f" % (step, loss))
        
    # Evaluation.
    if step % eval_step == 0 or step == 1:
        print("Evaluation...")
        sim = evaluate(get_embedding(x_test)).numpy()
        for i in range(len(eval_words)):
            top_k = 8  # number of nearest neighbors.
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = '"%s" nearest neighbors:' % eval_words[i]
            for k in range(top_k):
                log_str = '%s %s,' % (log_str, id2word[nearest[k]])
            print(log_str)

TypeError: ignored

In [66]:
new_word2id.keys()

dict_keys(['UNK', b'the', b'of', b'and', b'one', b'in', b'a', b'to', b'zero', b'nine', b'two', b'is', b'as', b'eight', b'for', b's', b'five', b'three', b'was', b'by', b'that', b'four', b'six', b'seven', b'with', b'on', b'are', b'it', b'from', b'or', b'his', b'an', b'be', b'this', b'which', b'at', b'he', b'also', b'not', b'have', b'were', b'has', b'but', b'other', b'their', b'its', b'first', b'they', b'some', b'had', b'all', b'more', b'most', b'can', b'been', b'such', b'many', b'who', b'new', b'used', b'there', b'after', b'when', b'into', b'american', b'time', b'these', b'only', b'see', b'may', b'than', b'world', b'i', b'b', b'would', b'd', b'no', b'however', b'between', b'about', b'over', b'years', b'states', b'people', b'war', b'during', b'united', b'known', b'if', b'called', b'use', b'th', b'system', b'often', b'state', b'so', b'history', b'will', b'up', b'while', b'where', b'city', b'being', b'english', b'then', b'any', b'both', b'under', b'out', b'made', b'well', b'her', b'e', b'nu

In [54]:
types =  [type(key) for (key, value) in word2id.items()]